# Data Mining Challenge

#### Jiyeon Song (A17151720)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import re


%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

Assignment Submission Page: https://www.kaggle.com/competitions/ucsd-dsc190-wi23-introduction-to-data-mining/

In [10]:
df = pd.read_csv("train.csv")
df

C:\Users\jyson\AppData\Local\Temp\ipykernel_29404\1862463380.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train.csv")


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,22267382,Modern and Cozy Large Studio in Brooklyn,Modern large studio with new amenities and app...,Our place is a little quiet sanctuary in the h...,Modern large studio with new amenities and app...,none,"BAM, Barclays, Brooklyn City Point, Fort Green...",NaN,"Subway: 2,3,4,5,A,C,B,Q,G",Washer/Dryer Dishwasher Internet Gym Roof Top ...,...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.59
1,2473861,Royal Harlem TRIPLEX Home 5 Beds,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,Harlem is back and so gorgeous! Visit and expl...,none,HARLEM is a piece of real NY history overflowi...,HARLEM RESTAURANTS Red Rooster Harlem -- excel...,PUBLIC TRANSPORTATION: Conveniently near all p...,The WHOLE ENTIRE HOUSE,...,9.0,9.0,9.0,t,f,moderate,f,f,3,2.47
2,25079703,Sunny East Village Studio,"Clean, hip and well designed sun drenched East...",This is a rare East Village studio with it's h...,"Clean, hip and well designed sun drenched East...",none,East Village is one of the last remaining neig...,NaN,NaN,You'll have access to the entire space - it's ...,...,10.0,10.0,10.0,f,f,moderate,f,f,1,0.89
3,9342478,"Beautiful, airy, light-filled room","Private, spacious, comfortable room in 2-bed f...","Big closet, two big windows, tall ceiling and ...","Private, spacious, comfortable room in 2-bed f...",none,One block from Morgan L stop. Super cool area....,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
4,4866426,Private Room in Prime Brooklyn Spot,"Comfy, quiet and big private room in a three b...",This big old apartment that we love and take c...,"Comfy, quiet and big private room in a three b...",none,I absolutely love this neighborhood - right at...,Just a note about the space: The window in you...,Super convenient to almost all subway lines. A...,Your room has a very comfortable queen sized b...,...,10.0,10.0,10.0,f,f,flexible,f,f,1,3.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33533,3958032,Private Room Amazing View in EV,"Private room in cozy, sun-drenched & plant fil...",NaN,"Private room in cozy, sun-drenched & plant fil...",none,NaN,NaN,NaN,NaN,...,9.0,9.0,8.0,f,f,strict_14_with_grace_period,f,f,1,0.44
33534,7987209,Floor 35th in the heart of New York,This is a beautiful four bedroom and three bat...,This is a Duplex Apartment (2 floors) Main Flo...,This is a beautiful four bedroom and three bat...,none,LOCATION LOCATION LOCATION. Walk to Time Squar...,NaN,Apartment is located at 70 W 45th Street (betw...,Access to the unit is through elevators.,...,10.0,10.0,9.0,f,f,strict_14_with_grace_period,f,f,1,2.28
33535,796128,Cozy room for rent in duplex house,NaN,Room for rent in duplex house located in Astor...,Room for rent in duplex house located in Astor...,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
33536,17281121,"Beautiful 1 bedroom apart, Washington Heights",Very spacious bright beautiful apartment in Wa...,"Very spacious living room, bright kitchen with...",Very spacious bright beautiful apartment in Wa...,none,Very close to Times Square but a quiet residen...,NaN,Very Close to subway lines A and 1 trains. Les...,"Living room, bedroom, shared kitchen and bathr...",...,10.0,10.0,10.0,f,f,moderate,f,f,3,0.10


## GOAL
#### Predicting the BEST price

## PLANS

#### what features can I use?
* **id(int)**
* description(str) - not sure
* neighborhood_overview(str) - not sure
* transit(str) - not sure
* access(str) - not sure
* **host_is_superhost(bool)** - true & false
* neighbourhood_group_cleansed(str) - 
* **property_type(str)** - category
* **room_type(str)** - category
* accommodates(int) - guest nums
* **bathrooms(int)**
* **bedrooms(int)**
* **beds(int)**
* **amanities(str)** - can categorize
* **price(int)**
* minimum_nights(int) - not sure
* **review_scores_rating(float) - out of 100**
* review_scores_accuracy(float) - out of 10
* **review_scores_cleanliness(float) - out of 10**
* **review_scores_location(float) - out of 10**
* cancellation_policy(str) - category
* reviews_per_month(int)


In [38]:
df['cancellation_policy'].dtypes

dtype('O')

In [11]:
from math import isnan

# to convert into float
def to_float(s):
    try:
        ret = float(s)
    except:
        ret = -1
    if isnan(ret):
        ret = -1
    return ret

# to convert into integer
def to_int(s):
    try:
        ret = int(s)
    except:
        ret = -1
    return ret

# to group by its categories
categories = [x for x in list(set(df['neighbourhood_group_cleansed'])) if type(x) is str]
# print(len(categories))

# choose features that I am going to use
def create_feature(row):
    guest_num = to_int(row.guests_included)
    guest_cap = to_int(row.extra_people)
    bedrooms = to_int(row.bedrooms)
    bathrooms = to_int(row.bathrooms)
    review = to_float(row.review_scores_rating)
    one_hot = [int(row.neighbourhood_group_cleansed == category) for category in categories]
    
    return [guest_num, guest_cap, bedrooms, bathrooms, review] + one_hot

# set features(list above) and target(price)
train_X, train_y = [], []
for (idx, row) in df.iterrows():
    price = float(row.price)
    features = create_feature(row)

    train_X.append(features)
    train_y.append(price)

print(len(train_X), len(train_y))

5
33538 33538


In [44]:
from sklearn.pipeline import Pipeline, make_union
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['']), 
        ('cat', OneHotEncoder(), ['']), 
        ('imp', SimpleImputer(strategy='most_frequent'), [''])], remainder='passthrough')

# # Define the first pipeline for text data
# text_pipeline = Pipeline([
#     ('tfidf', TfidfVectorizer()), 
#     ('clf', LogisticRegression())
#     ])

# Define the second pipeline for numerical data
pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('classifier', LogisticRegression())
    ])

# # Combine the pipelines using make_union
# combined_pipeline = make_union(text_pipeline, numerical_pipeline)

# # Train the pipeline on the training data
pipeline.fit(train_X, train_y)

TypeError: All estimators should implement fit and transform. 'Pipeline(steps=[('clf', LinearRegression())])' (type <class 'sklearn.pipeline.Pipeline'>) doesn't

### Testing for Overfitting

In [39]:
from sklearn.model_selection import train_test_split
# X = df.drop('price', axis=1)
# y = df['price']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# testing train set
pipeline.fit(train_X, train_y)
predict_train = pipeline.predict(train_X)
# rmse
def rmse(actual, pred):
    return np.sqrt(np.mean((actual - pred)**2))
# test set should have less root mean squared error
rmse(train_y, predict_train)#, rmse(y_test, predict_test)


NameError: name 'combined_pipeline' is not defined

## TEST

In [33]:
test_df = pd.read_csv('test.csv')
test_df

C:\Users\jyson\AppData\Local\Temp\ipykernel_29404\2079203658.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('test.csv')


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,19307997,Super Lux 2BR in Downtown Manhattan,Prepare to be WOWED! This spectacularly bright...,"Top of the line Wolf and Sub-Zero appliances, ...",Prepare to be WOWED! This spectacularly bright...,none,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,flexible,f,f,1,NaN
1,20176193,Vintage Eclectic Brownstone Pad in Brooklyn,"Ideal for romantic, creative types, this is an...","Not your typical New York abode, my apartment ...","Ideal for romantic, creative types, this is an...",none,Bed Stuy is a diverse historic neighborhood wi...,This is an actual unique living experience whe...,Close to buses and subways there is also free ...,"Entrance hallway, living room, bedroom, kitche...",...,10.0,10.0,10.0,f,f,flexible,f,f,1,1.48
2,19485371,Spacious Harlem Hideaway,"Postive Vibes . This is our Harlem tree house,...",The private room is very spacious and cozy. Th...,"Postive Vibes . This is our Harlem tree house,...",none,You are in a Cultural Haven full of restaurant...,We also keep cucumber water in the fridge feel...,"Train, uber or a taxi. (Extremely taxi accessi...","Private Room, Kitchen And Bathroom",...,10.0,10.0,10.0,t,f,flexible,f,f,1,0.37
3,13079990,Spacius private room in Brooklyn,"Newly renovated apartment, its a 3 bedroom apa...","3 bedroom apartment, 1 full bathroom, living r...","Newly renovated apartment, its a 3 bedroom apa...",none,NaN,NaN,"There is the Mta 3 train Sutter stop, also the...",NaN,...,9.0,8.0,9.0,f,f,flexible,f,f,1,0.23
4,22339757,*Dg) Delightful Private Room 20 min to Manhattan,Hi my home is only 2 blocks from the subway st...,NaN,Hi my home is only 2 blocks from the subway st...,none,NaN,NaN,NaN,NaN,...,8.0,8.0,8.0,t,f,strict_14_with_grace_period,f,f,9,1.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17332,22325617,Charming room in Brooklyn,Charming newly renovated 2 bed apartment in Be...,NaN,Charming newly renovated 2 bed apartment in Be...,none,NaN,NaN,"Though street parking is available, space is 2...",NaN,...,NaN,NaN,NaN,t,f,flexible,f,f,1,NaN
17333,8372650,Luxurious 1BR in Herald Square,"-Apartment includes: Hardwood Floors, High Cei...",- ONE proper bed that can sleep 2. - ONE plush...,"-Apartment includes: Hardwood Floors, High Cei...",none,Smack in the middle of Manhattan. Near the Emp...,NaN,NaN,NaN,...,NaN,NaN,NaN,f,f,strict_14_with_grace_period,f,f,1,NaN
17334,3812554,"Master Bedrm, Steam Shr/Jacuzzi, FH","Elegant room w/AC, King Bed, Jacuzzi and Steam...",This stately red brick federal on a quiet stre...,"Elegant room w/AC, King Bed, Jacuzzi and Steam...",none,It's easy to enjoy all that NYC has to offer f...,Guests are expected to respect others' needs f...,A three block walk to the Forest Hills/71st Rd...,Guests have access to the common areas of the ...,...,10.0,10.0,10.0,f,f,moderate,f,f,3,1.32
17335,18891508,Private rooms starting at $67 a night per person.,Family friendly neighborhood. Caribbean settin...,You have a choice of 1 of 4 bedrooms. Room#4 c...,Family friendly neighborhood. Caribbean settin...,none,The neighborhood has easy access to Manhattan....,Our place can also be available for $249 a nig...,Buses are 1 block away. A 10 minute ride in th...,"Coffee, tea, cocoa and various juices are prov...",...,10.0,9.0,10.0,f,f,moderate,f,f,4,0.29


In [34]:
test_ids, test_X = [], []
for (idx, row) in test_df.iterrows():
    feature = create_feature(row)
    test_ids.append(row.id)
    test_X.append(feature)

# Test the pipeline on the test data
test_y = combined_pipeline.predict(test_X)

NameError: name 'pipeline' is not defined

In [25]:
print(len(test_y))

17337


In [27]:
output_df = pd.DataFrame()
output_df['Id'] = test_ids
output_df['Predicted'] = test_y
# output_df.to_csv('my_baseline.csv', index = False)